In [ ]:
### Qué es esto???

%pip install lazypredict

In [ ]:
### 

import seaborn as sns
sns.set_theme(style="whitegrid")
%matplotlib inline
import requests
from pprint import pprint
!pip install pdfx
import pdfx

from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from google.colab import data_table
data_table.enable_dataframe_formatter()
data_table.disable_dataframe_formatter()
# ANOVA feature selection for numeric input and categorical output
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


In [ ]:
## User defined functions to get more accurate values than describe()

import math
def rounder(n):
    """
    Round up to the first digit that is not zero
    """
    if n == 0:
        return n
    else:
        k = 1 - int(math.log10(n))
        return round(n, 1 if n > 1 else k)

def describer(dataframe):
    """
    Description of dataframe
    """
    desc_df = round(dataframe.describe(include="all"),2).T
    desc_df["unique"]=dataframe.nunique()
    desc_df["NullAny"]= dataframe.isnull().any()
    desc_df["NullSum"]=dataframe.isnull().sum()
    desc_df["NullPct"]=(dataframe.isnull().sum()/len(dataframe)*100).apply(rounder)
    desc_df["dtypes"]=dataframe.dtypes
    desc_df["dtypes"] = desc_df["dtypes"].astype('category')
    desc_df["count"] = desc_df["count"].astype(int)
    desc_df.sort_values(by="dtypes", inplace=True)
    return(desc_df)

In [ ]:
## Import test dataset
test_x = pd.read_csv('https://challenges-asset-files.s3.us-east-2.amazonaws.com/data_sets/Data-Science/4+-+events/SchneiderElectricES22/final/test_x.csv', index_col="test_index")

In [ ]:
## Here we import the train 'CSV's, clean a bit and fuse them as one

df_1 = pd.read_csv('/content/drive/MyDrive/02-Schneider-Electric-Hack/Data-Science-SE/data/train1.csv')
df_2 = pd.read_csv('/content/drive/MyDrive/02-Schneider-Electric-Hack/Data-Science-SE/data/train2.csv', sep=";")
df_2.drop_duplicates(inplace=True)
df_csv = pd.concat([df_1, df_2], ignore_index=True)

In [ ]:
## Time to create a nes column whith proper datetime format for further manipulation
df_csv["date"] = pd.to_datetime(df_csv["reportingYear"].astype("str")+"-"+df_csv["MONTH"].astype("str")+"-"+df_csv["DAY"].astype("str"))

In [ ]:
## Same as 'CSV's, but with JSON's

# import json 1

api_url = "http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/first"
response = requests.get(api_url)
data = response.json()
df_3 = pd.DataFrame(data)
df_3.drop(columns=[''], inplace=True)
# df_3["date"] = pd.to_datetime(df_3["reportingYear"]+"-"+df_3["MONTH"]+"-"+df_3["DAY"])


api_url = "http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/second"
response = requests.get(api_url)
data = response.json()
df_4 = pd.DataFrame(data)
df_4.drop(columns=[''], inplace=True)
# df_4["date"] = pd.to_datetime(df_3["reportingYear"]+"-"+df_3["MONTH"]+"-"+df_3["DAY"])
api_url = "http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/third"
response = requests.get(api_url)
data = response.json()
df_5 = pd.DataFrame(data)
df_5.drop(columns=[''], inplace=True)
# df_5["date"] = pd.to_datetime(df_3["reportingYear"]+"-"+df_3["MONTH"]+"-"+df_3["DAY"])
df_json = pd.concat([df_3,df_4,df_5])

df_json[test_x.select_dtypes('int').columns] = df_json[test_x.select_dtypes('int').columns].astype("int64")
df_json[test_x.select_dtypes('float64').columns] = df_json[test_x.select_dtypes('float64').columns].astype("float64")
df_json[test_x.select_dtypes('object').columns] = df_json[test_x.select_dtypes('object').columns].astype("object")

In [ ]:
df_3[test_x.select_dtypes('int').columns] = df_3[test_x.select_dtypes('int').columns].astype("int64")
df_3[test_x.select_dtypes('float64').columns] = df_3[test_x.select_dtypes('float64').columns].astype("float64")
df_3[test_x.select_dtypes('object').columns] = df_3[test_x.select_dtypes('object').columns].astype("object")

In [ ]:
df = df_3[test_x.columns]

In [ ]:
def parse_values(x):
    if x =="Nitrogen oxides (NOX)":
       return 0
    elif x =="Carbon dioxide (CO2)":
       return 1
    else:
       return 2

df['target'] = df_3['pollutant'].apply(parse_values)

In [ ]:
df.drop(columns = df.columns[df.nunique() == 1], inplace = True)

In [ ]:
df.drop(columns=['EPRTRSectorCode', "REPORTER NAME", "City", "FacilityInspireID", "EPRTRAnnexIMainActivityCode"], inplace=True)

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df.isnull().sum()

countryName                     0
eprtrSectorName                 0
EPRTRAnnexIMainActivityLabel    0
facilityName                    0
reportingYear                   0
MONTH                           0
DAY                             0
max_wind_speed                  0
avg_wind_speed                  0
min_wind_speed                  0
max_temp                        0
avg_temp                        0
min_temp                        0
DAY WITH FOGS                   0
CITY ID                         0
target                          0
dtype: int64

In [ ]:
# Ordinal encoding 
encoder = OrdinalEncoder()
df[df.select_dtypes('object').columns]= encoder.fit_transform(df[df.select_dtypes('object').columns])

In [ ]:
df[df.select_dtypes('number').columns]

,countryName,eprtrSectorName,EPRTRAnnexIMainActivityLabel,facilityName,reportingYear,MONTH,DAY,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,CITY ID,target
0,20.00,1.00,6.00,2131.00,2020,3,2,11.02,15.69,20.90,13.26,12.33,14.70,1,973.00,1
1,11.00,4.00,48.00,3947.00,2019,2,3,14.51,18.40,23.24,4.53,5.11,9.22,2,3152.00,0
2,31.00,2.00,62.00,1022.00,2009,9,1,20.26,21.86,23.96,10.67,10.21,14.72,12,2668.00,0
3,15.00,3.00,36.00,3487.00,2014,10,17,18.28,19.20,26.70,7.10,9.50,13.58,1,1904.00,2
4,10.00,8.00,53.00,431.00,2015,9,6,13.76,18.28,24.77,9.89,11.59,14.01,0,3128.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9495,31.00,8.00,56.00,1860.00,2019,10,8,15.41,21.28,22.65,3.17,4.67,7.16,6,2673.00,2
9496,14.00,2.00,62.00,1461.00,2009,6,18,15.39,21.18,25.05,6.68,8.71,10.11,0,3167.00,0
9497,28.00,2.00,62.00,531.00,2011,5,25,10.22,14.07,18.28,16.03,16.27,20.33,2,625.00,0
9498,31.00,2.00,62.00,2542.00,2011,5,23,11.20,16.60,21.47,16.43,15.61,20.22,12,2668.00,2


In [ ]:
# Matrix
X = df[df.select_dtypes('number').columns].drop(columns=['target'])
# Vector
y = df['target']

In [ ]:
X

,countryName,eprtrSectorName,EPRTRAnnexIMainActivityLabel,facilityName,reportingYear,MONTH,DAY,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,CITY ID
0,20.00,1.00,6.00,2131.00,2020,3,2,11.02,15.69,20.90,13.26,12.33,14.70,1,973.00
1,11.00,4.00,48.00,3947.00,2019,2,3,14.51,18.40,23.24,4.53,5.11,9.22,2,3152.00
2,31.00,2.00,62.00,1022.00,2009,9,1,20.26,21.86,23.96,10.67,10.21,14.72,12,2668.00
3,15.00,3.00,36.00,3487.00,2014,10,17,18.28,19.20,26.70,7.10,9.50,13.58,1,1904.00
4,10.00,8.00,53.00,431.00,2015,9,6,13.76,18.28,24.77,9.89,11.59,14.01,0,3128.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9495,31.00,8.00,56.00,1860.00,2019,10,8,15.41,21.28,22.65,3.17,4.67,7.16,6,2673.00
9496,14.00,2.00,62.00,1461.00,2009,6,18,15.39,21.18,25.05,6.68,8.71,10.11,0,3167.00
9497,28.00,2.00,62.00,531.00,2011,5,25,10.22,14.07,18.28,16.03,16.27,20.33,2,625.00
9498,31.00,2.00,62.00,2542.00,2011,5,23,11.20,16.60,21.47,16.43,15.61,20.22,12,2668.00


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Call train_test_split on the data and capture the results
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 6858)

## Lazy Predict helps build a lot of basic models without much code and helps understand which models works better without any parameter tuning

In [ ]:
# LazyClassifier Instance and fiting data

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)


In [ ]:
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 29/29 [00:47<00:00,  1.62s/it]


### Check which model did better on the dataset

In [ ]:
models

NameError: ignored

## Analize Classifier Function

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, euclidean_distances
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, recall_score, plot_confusion_matrix
from sklearn.metrics import precision_score, f1_score, classification_report, balanced_accuracy_score

In [ ]:
def analize_models(model, name, X_train, X_test, y_train, y_test):
    
    model.fit(X_train,y_train)    
    y_pred = model.predict(X_test)
    conf_matrix = confusion_matrix(y_test,y_pred,normalize='true')

    scores = {'model': str(name),
            'Accuracy':accuracy_score(y_test,y_pred),
            'B-Accuracy':balanced_accuracy_score(y_test,y_pred),
            'Roc_AUC':roc_auc_score(y_test,y_pred),
            'F1 Score':f1_score(y_test,y_pred, average='macro'),
            'Precision':precision_score(y_test,y_pred),
            'Recall':recall_score(y_test,y_pred)}

    print(model)
    print('')
    print(f'Accuracy : {accuracy_score(y_test,y_pred)}')
    print(f'B-Accuracy :{balanced_accuracy_score(y_test,y_pred)}')
    print(f'ROC - AUC : {roc_auc_score(y_test,y_pred)}')
    print(f'F1 Score : {f1_score(y_test,y_pred, average="macro")}')
    print(f'Precision : {precision_score(y_test,y_pred)}')
    print(f'Recall : {recall_score(y_test,y_pred)}')
    print('')
    print(conf_matrix)
    print('')

    plot_confusion_matrix(model, X_test, y_test, normalize='true')
    

    return scores

In [ ]:
models.head(15).sort_values(by='Balanced Accuracy', ascending=False)

NameError: ignored

In [ ]:


# Set the figure size
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True

# Plot the dataframe
ax = df[['max_wind_speed', 'avg_wind_speed','min_wind_speed']].plot(kind='box', title='boxplot')

# Display the plot
plt.show()

In [ ]:
ax = df[['max_temp', 'avg_temp', 'min_temp']].plot(kind='box', title='boxplot')

# Display the plot
plt.show()

In [ ]:
ax = df[["DAY WITH FOGS"]].plot(kind='box', title='boxplot')

# Display the plot
plt.show()

In [ ]:
describer(df[df.select_dtypes('object').columns])

,count,unique,top,freq,NullAny,NullSum,NullPct,dtypes
countryName,9500,32,United Kingdom,1287,False,0,0.0,object
eprtrSectorName,9500,9,Energy sector,3610,False,0,0.0,object
EPRTRAnnexIMainActivityLabel,9500,69,Thermal power stations and other combustion in...,3171,False,0,0.0,object
facilityName,9500,4497,Enel Produzione S.p.A.,33,False,0,0.0,object
CITY ID,9500,3277,cfab1ba8c67c7c838db98d666f02a132,279,False,0,0.0,object


In [ ]:
describer(df[df["facilityName"]=="Enel Produzione S.p.A."])

In [ ]:
df[df["CITY ID"]=="cfab1ba8c67c7c838db98d666f02a132"].reportingYear.value_counts()

In [ ]:
describer(df[df["CITY ID"]=="cfab1ba8c67c7c838db98d666f02a132"])

In [ ]:
df[df["CITY ID"]=="cfab1ba8c67c7c838db98d666f02a132"]

In [ ]:
df[df["facilityName"]=="Enel Produzione S.p.A."]

In [ ]:
# df.info()
# df_csv.describe()
# df_json.describe()
# df.describe()
# statistics.stdev(df["DAY WITH FOGS"])
# # Import Z-score function
# from scipy.stats import zscore

# # Define the SD threshold
# thresh = 3

# SD_outliers = df[["DAY WITH FOGS"]].apply(lambda x: np.abs(zscore(x, nan_policy = 'omit')) > thresh).any(axis=1)
# df[["DAY WITH FOGS"]].index[SD_outliers]


# # List of all rows as `True` or `False` depending on if they have a value above the threshold or not
# SD_outliers = X_train.apply(lambda x: np.abs(zscore(x, nan_policy = 'omit')) > thresh).any(axis=1)


# # Drop (inplace) rows that have True in SD_Norm
# X_train.drop(X_train.index[SD_outliers], inplace = True)

In [ ]:

# generate dataset
X, y = make_classification(n_samples=100, n_features=20, n_informative=2)


In [ ]:
y=df['target'].values

In [ ]:
y

array([1, 0, 0, ..., 0, 2, 0])

In [ ]:

print(y.shape)

(9500,)


In [ ]:
y

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0])

In [ ]:
X = df[df.select_dtypes('number').columns].drop(columns=['target']).values

In [ ]:

print(X.shape)

(9500, 10)


In [ ]:
X

array([[2.02000000e+03, 3.00000000e+00, 2.00000000e+00, ...,
        1.23262344e+01, 1.46968954e+01, 1.00000000e+00],
       [2.01900000e+03, 2.00000000e+00, 3.00000000e+00, ...,
        5.11468848e+00, 9.21900340e+00, 2.00000000e+00],
       [2.00900000e+03, 9.00000000e+00, 1.00000000e+00, ...,
        1.02088371e+01, 1.47154651e+01, 1.20000000e+01],
       ...,
       [2.01100000e+03, 5.00000000e+00, 2.50000000e+01, ...,
        1.62732343e+01, 2.03307927e+01, 2.00000000e+00],
       [2.01100000e+03, 5.00000000e+00, 2.30000000e+01, ...,
        1.56109271e+01, 2.02226021e+01, 1.20000000e+01],
       [2.01000000e+03, 9.00000000e+00, 3.00000000e+00, ...,
        3.16269028e+00, 6.11534364e+00, 2.00000000e+00]])

In [ ]:
X

array([[-0.2404983 , -0.26683029, -1.80541662, ..., -0.76323861,
         0.23550136, -0.13569883],
       [-0.65706189,  0.18928333, -0.11946401, ..., -0.26040084,
         0.8084616 , -0.10428183],
       [ 0.46988543,  1.4171953 ,  0.96771505, ..., -2.30756078,
        -1.73304007, -0.16243867],
       ...,
       [ 1.15651429, -0.77567662, -0.2899093 , ...,  0.68113182,
        -0.70791982, -1.53333665],
       [ 0.17009692, -1.30536522,  0.09645506, ..., -0.05888199,
         1.10469808,  0.77414048],
       [ 1.2048644 ,  0.37549435, -1.24179862, ..., -1.78900392,
        -1.05215281, -1.59849675]])

In [ ]:

# define feature selection
fs = SelectKBest(score_func=f_classif, k=9)


In [ ]:
fs

SelectKBest(k=3)

In [ ]:
# apply feature selection
X_selected = fs.fit_transform(X, y)
X_selected

array([[2.02000000e+03, 3.00000000e+00, 2.00000000e+00, ...,
        1.23262344e+01, 1.46968954e+01, 1.00000000e+00],
       [2.01900000e+03, 2.00000000e+00, 3.00000000e+00, ...,
        5.11468848e+00, 9.21900340e+00, 2.00000000e+00],
       [2.00900000e+03, 9.00000000e+00, 1.00000000e+00, ...,
        1.02088371e+01, 1.47154651e+01, 1.20000000e+01],
       ...,
       [2.01100000e+03, 5.00000000e+00, 2.50000000e+01, ...,
        1.62732343e+01, 2.03307927e+01, 2.00000000e+00],
       [2.01100000e+03, 5.00000000e+00, 2.30000000e+01, ...,
        1.56109271e+01, 2.02226021e+01, 1.20000000e+01],
       [2.01000000e+03, 9.00000000e+00, 3.00000000e+00, ...,
        3.16269028e+00, 6.11534364e+00, 2.00000000e+00]])

In [ ]:

print(X_selected.shape)

(9500, 3)


In [ ]:
X_selected[0]

array([2.02000000e+03, 3.00000000e+00, 2.00000000e+00, 1.10193287e+01,
       1.56864502e+01, 1.32568160e+01, 1.23262344e+01, 1.46968954e+01,
       1.00000000e+00])

In [ ]:
 df[df.select_dtypes('number').columns].drop(columns=['target'])

,reportingYear,MONTH,DAY,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS
0,2020,3,2,11.019329,15.686450,20.899762,13.256816,12.326234,14.696895,1
1,2019,2,3,14.512395,18.396180,23.243403,4.528859,5.114688,9.219003,2
2,2009,9,1,20.262171,21.856210,23.956529,10.669133,10.208837,14.715465,12
3,2014,10,17,18.283547,19.195731,26.696266,7.095682,9.502302,13.582024,1
4,2015,9,6,13.759408,18.277089,24.768933,9.886774,11.585550,14.006226,0
...,...,...,...,...,...,...,...,...,...,...
9495,2019,10,8,15.413159,21.278369,22.654744,3.172374,4.669928,7.163001,6
9496,2009,6,18,15.391686,21.184836,25.052323,6.677630,8.710501,10.114623,0
9497,2011,5,25,10.223493,14.072257,18.279947,16.032472,16.273234,20.330793,2
9498,2011,5,23,11.199091,16.597921,21.472440,16.425340,15.610927,20.222602,12


In [ ]:
df['target']

In [ ]:
df_3['pollutant']

In [ ]:
test_x.select_dtypes('object').columns

Index(['countryName', 'eprtrSectorName', 'EPRTRAnnexIMainActivityCode',
       'EPRTRAnnexIMainActivityLabel', 'FacilityInspireID', 'facilityName',
       'City', 'targetRelease', 'CONTINENT', 'REPORTER NAME', 'CITY ID'],
      dtype='object')

In [ ]:
df_json[df_csv.select_dtypes('int').columns] = df_csv[df_csv.select_dtypes('int').columns].astype("int64")
df_json[df_csv.select_dtypes('float').columns] = df_json[df_csv.select_dtypes('float').columns].astype("float64")

In [ ]:
cols = list(set(df_csv.columns).intersection(df_json.columns))
cols.sort()

In [ ]:

df = pd.concat([df_csv[cols],df_json[cols]])

In [ ]:
df[["reportingYear", "MONTH", "DAY"]] = df[["reportingYear", "MONTH", "DAY"]].astype("int").astype("str")

In [ ]:
df["date"] = pd.to_datetime(df["reportingYear"]+"-"+df["MONTH"]+"-"+df["DAY"])

In [ ]:
describer(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  app.launch_new_instance()


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max,NullAny,NullSum,NullPct,dtypes
DAY,61853,28,NaN,NaN,NaT,NaT,14.5,8.13,1.0,7.0,14.0,22.0,28.0,False,0,0.0,int64
DAY WITH FOGS,61853,20,NaN,NaN,NaT,NaT,2.21,3.73,0.0,0.0,1.0,2.0,19.0,False,0,0.0,int64
MONTH,61853,12,NaN,NaN,NaT,NaT,6.46,3.46,1.0,3.0,6.0,9.0,12.0,False,0,0.0,int64
reportingYear,61853,14,NaN,NaN,NaT,NaT,2012.98,3.86,2007.0,2010.0,2013.0,2016.0,2020.0,False,0,0.0,int64
min_wind_speed,61853,57060,NaN,NaN,NaT,NaT,22.52,3.06,15.03,20.35,22.54,24.71,29.93,False,0,0.0,float64
min_temp,61853,57060,NaN,NaN,NaT,NaT,13.45,5.22,0.89,9.9,13.69,17.28,24.9,False,0,0.0,float64
max_wind_speed,61853,57060,NaN,NaN,NaT,NaT,15.52,3.07,8.01,13.33,15.51,17.72,22.99,False,0,0.0,float64
max_temp,61853,57060,NaN,NaN,NaT,NaT,9.46,5.21,-3.14,5.89,9.69,13.29,20.94,False,0,0.0,float64
avg_temp,61853,57060,NaN,NaN,NaT,NaT,10.45,5.08,-0.2,7.19,10.7,14.2,20.0,False,0,0.0,float64
avg_wind_speed,61853,57060,NaN,NaN,NaT,NaT,18.01,2.31,14.0,16.01,18.02,20.01,22.0,False,0,0.0,float64


KeyError: ignored

In [ ]:
df.Target.value_counts(normalize=True).round(2)

0    0.40
1    0.35
2    0.25
Name: Target, dtype: float64

In [ ]:
# Correlation between numeric columns
# ==============================================================================
def tidy_corr_matrix(corr_mat):
    
    # Function to convert a pandas correlation matrix to tidy format
    
    corr_mat = corr_mat.stack().reset_index()
    corr_mat.columns = ['variable_1','variable_2','r']
    corr_mat = corr_mat.loc[corr_mat['variable_1'] != corr_mat['variable_2'], :]
    corr_mat['abs_r'] = np.abs(corr_mat['r'])
    corr_mat = corr_mat.sort_values('abs_r', ascending=False)
    
    return corr_mat



In [ ]:
corr_matrix = df.select_dtypes(include=['float64', 'int']).corr(method='pearson')

tidy_corr_matrix(corr_matrix)

In [ ]:
# Heatmap matrix of correlations
# ==============================================================================
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))

corr= df.select_dtypes(include=['float64', 'int']).corr(method='pearson').corr()

# Getting the Upper Triangle of the co-relation matrix

matrix = np.triu(corr)

# using the upper triangle matrix as mask 
sns.heatmap(corr, 
            annot=True, 
            mask=matrix, 
            cmap=sns.diverging_palette(150, 275, s=80, l=55, n=9),
            annot_kws = {"size": 10})

ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation = 45,
    horizontalalignment = 'right',
)
ax.set_yticklabels(
    ax.get_yticklabels(),
    rotation = 0,
    horizontalalignment = 'right',
)
ax.tick_params(labelsize = 15)

fig.suptitle('Heatmap Correlation Matrix', fontsize = 30, fontweight = "bold")
#plt.savefig("../reports/figures/Heatmap_Matrix_Correlations.png")

In [ ]:
df_csv

In [ ]:
data_table.DataTable(describer(df_csv)[["unique"]])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  app.launch_new_instance()


,unique
DAY WITH FOGS,20
reportingYear,14
MONTH,12
DAY,28
min_temp,28559
avg_temp,28559
max_temp,28559
min_wind_speed,28559
avg_wind_speed,28559
max_wind_speed,28559


In [ ]:
data_table.DataTable(describer(test_x)[["unique"]])

,unique
MONTH,12
EPRTRSectorCode,9
DAY WITH FOGS,20
reportingYear,14
DAY,28
min_temp,24480
avg_temp,24480
max_temp,24480
min_wind_speed,24480
avg_wind_speed,24480


In [ ]:
data_table.DataTable(describer(df_3)[["unique"]])

,unique
,9500
reportingYear,14
pollutant,3
min_wind_speed,9500
min_temp,9500
max_wind_speed,9500
max_temp,9500
facilityName,4497
eprtrSectorName,9
countryName,32


In [ ]:
data_table.DataTable(describer(df))

In [ ]:
data_table.DataTable(describer(df)

In [ ]:
describer(test_x)

In [ ]:
df.shape

(61853, 24)

In [ ]:
data_table.DataTable(df, include_index=False, max_columns=24, max_rows=61853)

In [ ]:
data_table.DataTable(test_x, include_index=False, max_columns=22, max_rows=24480)

In [ ]:
df[["facilityName","FacilityInspireID"]]

In [ ]:
df["eprtrSectorName"].unique()

In [ ]:
df["countryName"].unique()

In [ ]:
df["EPRTRAnnexIMainActivityLabel"].unique()

In [ ]:
df["City"].unique()

In [ ]:
pdfx.PDFx("/content/drive/MyDrive/02-Schneider-Electric-Hack/Data-Science-SE/data/train6/pdfs81515.pdf").get_text()

In [ ]:
pdfx.PDFx("/content/drive/MyDrive/02-Schneider-Electric-Hack/Data-Science-SE/data/train6/pdfs81560.pdf").get_text()

In [ ]:
pdfx.PDFx("/content/drive/MyDrive/02-Schneider-Electric-Hack/Data-Science-SE/data/train6/pdfs81575.pdf").get_text()

In [ ]:
pdfx.PDFx("/content/drive/MyDrive/02-Schneider-Electric-Hack/Data-Science-SE/data/train6/pdfs81580.pdf").get_text()

In [ ]:
pdfx.PDFx("/content/drive/MyDrive/02-Schneider-Electric-Hack/Data-Science-SE/data/train6/pdfs81593.pdf").get_text()

In [ ]:

pdf = pdfx.PDFx("/content/drive/MyDrive/02-Schneider-Electric-Hack/Data-Science-SE/data/train6/pdfs81515.pdf")
pdf

Next, use the `get_text()` function to extract the text from the `pdf` object:

In [ ]:
text = pdf.get_text()
text

'REPORT CONTAMINACIÓN\n\nFACILITY NAME:\nFacilityInspireID: UK.SEPA/200000061.Facility\n\nFife Ethylene Plant\n\nCOUNTRY:\nCITY:\n\nUnited Kingdom\nCowdenbeath\n\nnº:\n\n81516\n\n81516\n\nCONTINENT:\n\nEUROPE\n\nEPRTRSectorCode:\n\n1\n\neprtrSectorName:\n\nEnergy sector\n\nMainActivityCode: 1(c)\n\ntargetRealase:\nemissions:\n\n6,8E+08\n\nAIR\n\npollutant:\n\nCarbon dioxide (CO2)\n\nDAY:\n\n12\n\nMONTH:\n\n4 YEAR:\n\n2019\n\nMETEOROLOGICAL CONDITIONS\n\nmax_wind_speed: 1,52E+16\n\nmin_wind_speed:\n\n2,06E+15 avg_wind_speed: 1,46E+16\n\nmax_temp:\n\n9,61E+15\n\nmin_temp:\n\n1,33E+16 avg_temp:\n\n8,69E+15\n\nDAYS FOG:\n\n19\n\nREPORTER NAME:Shawn Graham\n\nCITY_ID\n\n3c563ab0d76fc84128574b5da82f769a\n\n\x0c'

In [ ]:
train1.columns

Index(['countryName', 'eprtrSectorName', 'EPRTRAnnexIMainActivityLabel',
       'FacilityInspireID', 'facilityName', 'City', 'targetRelease',
       'pollutant', 'reportingYear', 'MONTH', 'DAY', 'CONTINENT',
       'max_wind_speed', 'avg_wind_speed', 'min_wind_speed', 'max_temp',
       'avg_temp', 'min_temp', 'DAY WITH FOGS', 'REPORTER NAME', 'CITY ID'],
      dtype='object')

In [ ]:
pprint(text)